# ODM 2025 – Exercise Sheet 2: Black-box Service

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class BlackBox:
    """
    This class implements a simple interface to the black-box service for the ODM course.
    """

    def __init__(self, token: int, endpoint: str = 'http://ls-stat-ml.uni-muenster.de:7300/'):
        self.endpoint = endpoint
        self.token = token

    def set_endpoint(self, endpoint: str):
        self.endpoint = endpoint

    def evaluate(self, objective: str, parameters: list) -> float:
        r = requests.post(url=self.endpoint + "compute/" + objective,
                          json={"parameters": [str(v) for v in parameters], "token": self.token})
        return float(r.json())

    def evaluate_gradient(self, objective: str, parameters: list) -> list:
        r = requests.post(url=self.endpoint + "compute_gradient/" + objective,
                          json={"parameters": [str(v) for v in parameters], "token": self.token})
        return r.json()

In [3]:
group_number = 11

bb = BlackBox(token = group_number)

## Evaluation Examples

With `bb.evaluate`, you can call the objective function (first argument) on any (two-dimensional) numerical vector (second argument):

In [4]:
y = bb.evaluate("Function1", [0.81, 0.04])
y

0.7234700000000002

With `bb.evaluate_gradient`, you get the gradient of the objective function instead:

In [5]:
grad = bb.evaluate_gradient("Function2", [0.33, 0.44])
grad

[-37.2316894976521, -51.36205235078961]

That is all there is to set up! Now you can use `bb.evaluate` and `bb.evaluate_gradient` to explore and optimize the different black-box functions ...

Import optimization algorithms from separate modules:

In [33]:
# Import gradient-based (indirect) optimization methods
from indirect_methods import sgd, momentum, rmsprop, adam

# Import derivative-free (direct) optimization methods
from direct_methods import coordinate_search, hooke_jeeves, nelder_mead

## Our Solution

In [ ]:
# Example: Optimize Function1 with direct search methods
x0 = [0.5, 0.5]  # Starting point

# Coordinate Search
x_coord, history_coord = coordinate_search(bb, "Function1", x0, step_size=0.1, max_iter=200)
print(f"Coordinate Search: x = {x_coord}, f(x) = {bb.evaluate('Function1', x_coord.tolist())}\n")

# Hooke & Jeeves
x_hj, history_hj = hooke_jeeves(bb, "Function1", x0, step_size=0.1, max_iter=200)
print(f"Hooke & Jeeves: x = {x_hj}, f(x) = {bb.evaluate('Function1', x_hj.tolist())}\n")

# Nelder-Mead
x_nm, history_nm = nelder_mead(bb, "Function1", x0, max_iter=200)
print(f"Nelder-Mead: x = {x_nm}, f(x) = {bb.evaluate('Function1', x_nm.tolist())}")

In [15]:
x0 = [.5, .5]  # Starting point

In [27]:
# SGD
x_sgd, history_sgd = sgd(bb, "Function1", x0, learning_rate=0.01, max_iter=200)
print(f"SGD Result: x = {x_sgd}, f(x) = {bb.evaluate('Function1', x_sgd.tolist())}")

SGD Result: x = [0.00584441 0.00584441], f(x) = 7.514578135635796e-05


In [26]:
# Momentum
x_momentum, history_momentum = momentum(bb, "Function1", x0, learning_rate=0.01, beta=0.9, max_iter=200)
print(f"Momentum Result: x = {x_momentum}, f(x) = {bb.evaluate('Function1', x_momentum.tolist())}")

Momentum Result: x = [-1.38083156e-05 -1.38083156e-05], f(x) = 4.1947307435996864e-10


In [25]:
# RMSProp
x_rmsprop, history_rmsprop = rmsprop(bb, "Function1", x0, learning_rate=0.01, beta=0.9, max_iter=200)
print(f"RMSProp Result: x = {x_rmsprop}, f(x) = {bb.evaluate('Function1', x_rmsprop.tolist())}")

Converged after 89 iterations
RMSProp Result: x = [1.37314017e-07 1.37314017e-07], f(x) = 4.1481306354842827e-14


In [24]:
# Adam
x_adam, history_adam = adam(bb, "Function1", x0, learning_rate=0.01, beta1=0.9, beta2=0.999, max_iter=200)
print(f"Adam Result: x = {x_adam}, f(x) = {bb.evaluate('Function1', x_adam.tolist())}")

Adam Result: x = [-1.09539843e-05 -1.09539843e-05], f(x) = 2.6397749727146206e-10


## Problem Landscape Analysis

Let's visualize the different functions to understand their characteristics and group them.

In [ ]:
def create_grid(x_range=(-2, 2), y_range=(-2, 2), resolution=50):
    """Create a grid for visualization."""
    x = np.linspace(x_range[0], x_range[1], resolution)
    y = np.linspace(y_range[0], y_range[1], resolution)
    X, Y = np.meshgrid(x, y)
    return X, Y

def evaluate_on_grid(bb, objective, X, Y):
    """Evaluate the objective function on a grid."""
    Z = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = bb.evaluate(objective, [X[i, j], Y[i, j]])
    return Z

In [29]:
def visualize_function_landscape(bb, objective, x_range=(-2, 2), y_range=(-2, 2), 
                                 resolution=50, optimizer_traces=None):
    """
    Create contour plot and 3D surface plot for a function.
    
    Args:
        bb: BlackBox instance
        objective: Function name
        x_range: Range for x-axis
        y_range: Range for y-axis
        resolution: Grid resolution
        optimizer_traces: Dict of {optimizer_name: history} to overlay traces
    """
    print(f"Evaluating {objective} on grid (this may take a moment)...")
    X, Y = create_grid(x_range, y_range, resolution)
    Z = evaluate_on_grid(bb, objective, X, Y)
    
    fig = plt.figure(figsize=(16, 6))
    
    # 3D Surface Plot
    ax1 = fig.add_subplot(131, projection='3d')
    surf = ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8, antialiased=True)
    ax1.set_xlabel('x1')
    ax1.set_ylabel('x2')
    ax1.set_zlabel('f(x)')
    ax1.set_title(f'{objective} - 3D Surface')
    fig.colorbar(surf, ax=ax1, shrink=0.5)
    
    # Contour Plot
    ax2 = fig.add_subplot(132)
    contour = ax2.contour(X, Y, Z, levels=20, cmap='viridis')
    contourf = ax2.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)
    ax2.set_xlabel('x1')
    ax2.set_ylabel('x2')
    ax2.set_title(f'{objective} - Contour Plot')
    fig.colorbar(contourf, ax=ax2)
    
    # Add optimizer traces if provided
    if optimizer_traces:
        colors = ['red', 'blue', 'green', 'orange', 'purple']
        for idx, (opt_name, history) in enumerate(optimizer_traces.items()):
            x_trace = [h[1][0] for h in history]
            y_trace = [h[1][1] for h in history]
            ax2.plot(x_trace, y_trace, 'o-', color=colors[idx % len(colors)], 
                    label=opt_name, markersize=3, linewidth=1.5, alpha=0.7)
            ax2.plot(x_trace[0], y_trace[0], 'x', color=colors[idx % len(colors)], 
                    markersize=10, markeredgewidth=2, label=f'{opt_name} start')
            ax2.plot(x_trace[-1], y_trace[-1], '*', color=colors[idx % len(colors)], 
                    markersize=12, label=f'{opt_name} end')
    
    # Convergence Plot
    ax3 = fig.add_subplot(133)
    if optimizer_traces:
        for idx, (opt_name, history) in enumerate(optimizer_traces.items()):
            iterations = [h[0] for h in history]
            values = [h[2] for h in history]
            ax3.plot(iterations, values, '-', color=colors[idx % len(colors)], 
                    label=opt_name, linewidth=2)
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('f(x)')
    ax3.set_title(f'{objective} - Convergence')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n{objective} Statistics:")
    print(f"  Min value on grid: {Z.min():.6f}")
    print(f"  Max value on grid: {Z.max():.6f}")
    print(f"  Mean value: {Z.mean():.6f}")
    print(f"  Std deviation: {Z.std():.6f}")

### Visualization: Compare Problem Landscapes Side-by-Side

In [ ]:
functions = ["Function1", "Function2", "Function3", "Function4", "Function5"]
x_range = (-2, 2)
y_range = (-2, 2)
resolution = 40


n_funcs = len(functions)
fig, axes = plt.subplots(2, n_funcs, figsize=(5*n_funcs, 10))

if n_funcs == 1:
    axes = axes.reshape(-1, 1)

X, Y = create_grid(x_range, y_range, resolution)

for idx, func in enumerate(functions):
    print(f"Evaluating {func} for comparison...")
    Z = evaluate_on_grid(bb, func, X, Y)
    
    # 3D surface
    ax1 = fig.add_subplot(2, n_funcs, idx+1, projection='3d')
    surf = ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8, antialiased=True)
    ax1.set_xlabel('x1', fontsize=8)
    ax1.set_ylabel('x2', fontsize=8)
    ax1.set_zlabel('f(x)', fontsize=8)
    ax1.set_title(f'{func}\n3D Surface', fontsize=10)
    ax1.tick_params(labelsize=6)
    
    # Contour plot
    ax2 = axes[1, idx]
    contour = ax2.contour(X, Y, Z, levels=20, cmap='viridis')
    contourf = ax2.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)
    ax2.set_xlabel('x1', fontsize=8)
    ax2.set_ylabel('x2', fontsize=8)
    ax2.set_title(f'{func}\nContour', fontsize=10)
    ax2.tick_params(labelsize=6)
    plt.colorbar(contourf, ax=ax2)

plt.tight_layout()
plt.show()